In [18]:
import pandas as pd
import pickle
import numpy as np
import re
import codecs

from gensim.models.phrases import Phrases , Phraser
from janome.tokenizer import Tokenizer

In [19]:
def judge(word):
    #print(word)
    word_list=word.split("_")
    word=word.replace("_","")
    
    tokens = tokenizer.tokenize(word)
 
    if len(tokens)==1 or len(tokens)==0 or re.search("\s+", word): #空白文字が含まれていたら除外
        return pd.Series(["",""])
    
    if word_list[0]=="年" or word_list[0]=="年度": #最初が「年」または「年度」であれば除外
        return pd.Series(["",""])
    
    if "," in word_list:
        return pd.Series(["",""])
 
    part0 = tokens[-1].part_of_speech.split(',')[0]
 
    if part0 == "助詞" or part0=="助動詞": #最後が助詞または助動詞であればその単語を削除
        del tokens[-1]
        del word_list[-1]
         
        word="".join(word_list)
         
        if len(tokens)==1:
            return pd.Series([word,""])
         
    for num, token in enumerate(tokens):
         
        surFace = token.surface 
        part0 = token.part_of_speech.split(',')[0]
        part1 = token.part_of_speech.split(',')[1]
        part2 = token.part_of_speech.split(',')[2]
        reading = token.reading
        baseForm = token.base_form
        inflForm = token.infl_form
         
 
        if num==0: 
            if part0 ==u"接頭詞": #最初が接頭詞であれば、次の単語へ
                continue 
         
        if part0 != u'名詞':
            return pd.Series([word,False])
        elif part0 =="名詞" and (part1 == u'数' or part2=="助数詞"): #数、助数詞が含まれていれば除外
            return pd.Series([word,False])
        else:
            continue
 
    return pd.Series([word,True]) 

In [20]:
#読み込み
import_data_pd = pickle.load(open("import_data_pd_max.sav","rb"))

In [28]:
for category_id in import_data_pd["category"].drop_duplicates():
    gramCnt = 5 
    words_judge_count = 10
    text = ""
    tokenizer = Tokenizer()
    corpus=[]


    for sentence in import_data_pd["text"][import_data_pd["category"] == category_id]:
        text += sentence
    text_split=text.split("。")


    for text_list in text_split:
        tokens = tokenizer.tokenize(text_list)
        corpus.append([token.surface for token in tokens])

    corpus_phrase=corpus

    for i in range(gramCnt-1):
        phrases = Phrases(corpus_phrase,min_count=5, threshold=10.0)
        phraser = Phraser(phrases)
        transformed=list(phraser[corpus_phrase])

        corpus_phrase = transformed


    words_df=pd.DataFrame()

    for sentence in corpus_phrase:
        for word in sentence:
            if word.find('_') >= 0:
                word_b = word.encode('cp932', "ignore")
                word = word_b.decode('cp932')
                words_df=words_df.append([word])

    words_df.columns=["複合語候補"]

    words_df = words_df.groupby("複合語候補")[["複合語候補"]].count().rename(columns={'複合語候補': '回数'}).sort_values(by="複合語候補",ascending=False).reset_index()

    words_df=(
        pd.concat([words_df, words_df["複合語候補"].apply(judge)], axis=1 )
        .rename(columns={0:"単語",1:"判定"}))

    words_judge=(words_df.groupby(["単語","判定"])["回数"].sum()
            .reset_index()
            .rename(columns={0:"回数"})
            .sort_values(by="回数", ascending=False))

    words_dic=words_judge[(words_judge["判定"]==True) & (words_judge["回数"] >= words_judge_count)]
    
    if len(words_dic) != 0:
        words_dic["paramater"]="-1,-1,1000,名詞,一般,*,*,*,*,%s,*,*"

        words_dic=pd.concat(
            [words_dic['単語'],words_dic['paramater']
             .str.split(',', expand=True)], axis=1)

        words_dic[9]=words_dic["単語"]

        words_dic.to_csv(str(category_id) + "_dic.csv", sep=",",index=False,header=False,encoding='cp932') 
        print(str(category_id))
    else:
        print("0件:" + str(category_id))

C:\Users\tamura\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


9050


C:\Users\tamura\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


5050


C:\Users\tamura\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

2050
0件:3500


C:\Users\tamura\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


3150


C:\Users\tamura\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


7150
0件:50
0件:1050
0件:5200
0件:3350


In [29]:
import os
import glob
import csv

# フォルダ中のパスを取得
All_Files = glob.glob('*dic.csv')

In [30]:
dic_list = []
for file in All_Files:
    try:
        print(file)
        dic_list.append(pd.read_csv(file, encoding="shift-jis", header=None))
    except pd.io.common.EmptyDataError:
        print("ERROR: {} is empty".format(file))
    
    
df = pd.concat(dic_list, sort=False)

1050_dic.csv
2050_dic.csv
3050_dic.csv
3100_dic.csv
3150_dic.csv
3200_dic.csv
3250_dic.csv
3300_dic.csv
3350_dic.csv
3400_dic.csv
3450_dic.csv
3500_dic.csv
3550_dic.csv
3600_dic.csv
3650_dic.csv
3700_dic.csv
3750_dic.csv
3800_dic.csv
4050_dic.csv
5050_dic.csv
50_dic.csv
ERROR: 50_dic.csv is empty
5100_dic.csv
5150_dic.csv
5250_dic.csv
6050_dic.csv
6100_dic.csv
7050_dic.csv
7100_dic.csv
7150_dic.csv
7200_dic.csv
8050_dic.csv
9050_dic.csv


In [31]:
dic_df = df.drop_duplicates().reset_index(drop=True)
dic_df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,天然ガス,-1,-1,1000,名詞,一般,*,*,*,*,天然ガス,*,*
1,中国石油化工,-1,-1,1000,名詞,一般,*,*,*,*,中国石油化工,*,*
2,国際石油開発帝石,-1,-1,1000,名詞,一般,*,*,*,*,国際石油開発帝石,*,*
3,ガス田,-1,-1,1000,名詞,一般,*,*,*,*,ガス田,*,*
4,持株会社,-1,-1,1000,名詞,一般,*,*,*,*,持株会社,*,*
5,総公司,-1,-1,1000,名詞,一般,*,*,*,*,総公司,*,*
6,東京都,-1,-1,1000,名詞,一般,*,*,*,*,東京都,*,*
7,売上高,-1,-1,1000,名詞,一般,*,*,*,*,売上高,*,*
8,電気工事,-1,-1,1000,名詞,一般,*,*,*,*,電気工事,*,*
9,創業者,-1,-1,1000,名詞,一般,*,*,*,*,創業者,*,*


In [32]:
#カンマが入っている複合語を削除する。
dic_df = dic_df.drop(dic_df[dic_df[0].str.contains(',')].index[0])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [33]:
dic_df.to_csv("dic_df.csv",sep=",",index=False,header=False,encoding='cp932') 